In [150]:
import pandas as pd
import requests
import json
from datetime import date
from credentials import *

In [119]:
def get_dates(start, end):
    return pd.Series(pd.date_range(start, end,freq='d').format()) # Format makes the output ans str variable.

In [60]:
def city_code(city_name):
    url = "https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/autosuggest/v1.0/US/USD/en-US/"

    params = {"query": city_name}

    headers = {'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
                      'x-rapidapi-key': x_rapidapi_key}

    response = requests.get(url, headers = headers, params = params)

    return response.json()["Places"][0]["PlaceId"]

In [107]:
def flight_prices(departure, arrival, date_start, date_end):

    departure_code = city_code(departure)
    arrival_code = city_code(arrival)
    url = f"https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/US/USD/en-US/{departure_code}/{arrival_code}/{date_start}/{date_end}"

    params = {"inboundpartialdate":{date_start}, "outboundpartialdate":{date_end}}

    #headers = {
    #'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
    #'x-rapidapi-key': "<introduce your RapidAPI key here>"}

    headers = {
    'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
    'x-rapidapi-key': x_rapidapi_key}

    response = requests.get(url, headers=headers, params=params)

    return response.json()

json_archive = flight_prices('Paris', 'London','2021-02-25','2021-02-26')

In [113]:
def flight_prices_start(departure, arrival, date_start):

    departure_code = city_code(departure)
    arrival_code = city_code(arrival)
    url = f"https://skyscanner-skyscanner-flight-search-v1.p.rapidapi.com/apiservices/browsequotes/v1.0/US/USD/en-US/{departure_code}/{arrival_code}/{date_start}"

    params = {"inboundpartialdate":{date_start}}

    #headers = {
    #'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
    #'x-rapidapi-key': "<introduce your RapidAPI key here>"}

    headers = {
    'x-rapidapi-host': "skyscanner-skyscanner-flight-search-v1.p.rapidapi.com",
    'x-rapidapi-key': x_rapidapi_key}

    response = requests.get(url, headers=headers, params=params)

    return response.json()

In [122]:
dates = get_dates('2021-02-25','2021-02-28')

In [123]:
json_archives = {}
for day in dates:
    json_ = flight_prices_start('Paris', 'London', day)
    json_archives[day] = json_      

In [157]:
for day in dates:
    my_json = json_archives[day]
    flights = my_json['Quotes']
    #print(flights)
    
    flight_price = []
    for flight in flights:
        flight_price.append(flight['MinPrice'])
        #if(flight['MinPrice'] )
        
        
    min_day_price = min(flight_price)
    print(str(min_day_price) + "$  --- " + str(flight['OutboundLeg']['DepartureDate']) + " --- " + )

157$  --- 2021-02-25T00:00:00
133$  --- 2021-02-26T00:00:00
153$  --- 2021-02-27T00:00:00
98$  --- 2021-02-28T00:00:00


In [158]:
for key in x.keys():
    all_info['date'] = key
    all_info['price'] = x[key]['Quotes'][0]['MinPrice']
    all_info['flight_company'] = x[key]['Carriers'][0]['Name']


NameError: name 'x' is not defined

In [131]:
json_archives[dates[0]]



{'Quotes': [{'QuoteId': 1,
   'MinPrice': 157,
   'Direct': False,
   'OutboundLeg': {'CarrierIds': [1218],
    'OriginId': 73097,
    'DestinationId': 65698,
    'DepartureDate': '2021-02-25T00:00:00'},
   'QuoteDateTime': '2021-02-23T15:59:00'},
  {'QuoteId': 2,
   'MinPrice': 241,
   'Direct': True,
   'OutboundLeg': {'CarrierIds': [838],
    'OriginId': 44759,
    'DestinationId': 65698,
    'DepartureDate': '2021-02-25T00:00:00'},
   'QuoteDateTime': '2021-02-23T15:59:00'}],
 'Carriers': [{'CarrierId': 838, 'Name': 'Air France'},
  {'CarrierId': 1218, 'Name': 'Iberia'}],
 'Places': [{'Name': 'Paris Charles de Gaulle',
   'Type': 'Station',
   'PlaceId': 44759,
   'IataCode': 'CDG',
   'SkyscannerCode': 'CDG',
   'CityName': 'Paris',
   'CityId': 'PARI',
   'CountryName': 'France'},
  {'Name': 'London Heathrow',
   'Type': 'Station',
   'PlaceId': 65698,
   'IataCode': 'LHR',
   'SkyscannerCode': 'LHR',
   'CityName': 'London',
   'CityId': 'LOND',
   'CountryName': 'United Kingdom

In [114]:
quotes = pd.json_normalize(json_archive['Quotes'])
carriers = pd.json_normalize(json_archive['Carriers'])
places = pd.json_normalize(json_archive['Places'])
currencies = pd.json_normalize(json_archive['Currencies'])

In [115]:
quotes.head()

,QuoteId,MinPrice,Direct,QuoteDateTime,OutboundLeg.CarrierIds,OutboundLeg.OriginId,OutboundLeg.DestinationId,OutboundLeg.DepartureDate
0,1,157,False,2021-02-23T15:59:00,[1218],73097,65698,2021-02-25T00:00:00
1,2,241,True,2021-02-23T15:59:00,[838],44759,65698,2021-02-25T00:00:00


In [116]:
carriers.head()

,CarrierId,Name
0,838,Air France
1,1218,Iberia


In [117]:
places.head()

,Name,Type,PlaceId,IataCode,SkyscannerCode,CityName,CityId,CountryName
0,Paris Charles de Gaulle,Station,44759,CDG,CDG,Paris,PARI,France
1,London Heathrow,Station,65698,LHR,LHR,London,LOND,United Kingdom
2,Paris Orly,Station,73097,ORY,ORY,Paris,PARI,France
3,Paris,City,3579769,NaN,PARI,NaN,PARI,France


In [118]:
currencies.head()

,Code,Symbol,ThousandsSeparator,DecimalSeparator,SymbolOnLeft,SpaceBetweenAmountAndSymbol,RoundingCoefficient,DecimalDigits
0,USD,$,",",.,True,False,0,2
